<a href="https://colab.research.google.com/github/comasya/segundapreentrega_comas/blob/main/Preentrega2_Comas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Optimización del stock mínimo de repuestos**

##IA: Generación de prompts

Febrero 2025
Realizado por: Yamil Comas
Comisión 71950

#**Descripción del requerimiento**

El proyecto se basa en una constructora que cuenta con un taller interno donde se realizan las reparaciones y servicios de vehículos livianos y pesados utilizados en las obras. La gestión de repuestos en este taller es un factor crítico para evitar demoras en los servicios y asegurar la operatividad de la flota.

Actualmente, uno de los principales desafíos es identificar de manera eficaz la alta rotación de repuestos/ fluidos y establecer un stock mínimo adecuado en depósito.
Los servicios de mantenimiento tienen intervalos establecidos:

* Vehículos y camiones: cada 10,000 km.
* Moto soldadoras, electro soldadoras, retroexcavadoras, niveladoras y tiende tubos: cada 250 horas de uso.
* Si un equipo ha estado parado por más de 6 meses, se realiza un servicio completo(filtros y fluido).

La falta de un sistema predictivo eficiente puede generar problemas de desabastecimiento, retrasos en los servicios y costos adicionales en compras urgentes.


#**Desarrollo del objetivo**

Se propone la utilización de prompts para el análisis del stock mínimo de repuestos, basándose en la rotación y los servicios programados de los equipos. La solución se dividirá en tres partes principales:
Análisis de repuestos:

1.   Identificación de la rotación de filtros (aceite, combustible, aire) y otros repuestos críticos(correas, mangueras de motor)
2.   Análisis de fluidos: Evaluación del consumo y reposición de aceites y líquido refrigerante.
3.   Análisis combinado: Relación entre repuestos y fluidos, identificando correlaciones y patrones de uso.

Los prompts serán diseñados para generar informes cada 60 días(según necesidad), permitiendo:

*   Sugerir cantidades mínimas recomendadas según el historial de rotación y solicitudes previas.
*   Clasificar repuestos por urgencia, compatibilidad y disponibilidad. Permitir al usuario elegir el formato de salida (tabla, lista priorizada o gráficos).
*   Incluir la visualización de imágenes de repuestos para facilitar su identificación y la búsqueda de alternativas.


In [40]:
pip install Groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 9.1 MB/s eta 0:00:00


In [41]:
import pandas as pd

In [42]:
# Leer el archivo Excel
df = pd.read_excel('/content/drive/MyDrive/IA PROMPTS/BD_PROYECTO.xlsx')

In [43]:
print(df.columns)
print(df.head())

Index(['CODIGO', 'NOMBRE_REPUESTO', 'STOCK_ACTUAL', 'CONSUMO_MENSUAL_PROMEDIO',
       'TIEMPO_REPOSICION', 'CATEGORIA'],
      dtype='object')
   CODIGO                   NOMBRE_REPUESTO  STOCK_ACTUAL  \
0    1001   CORREA  5801378010 IVECO 170E22             6   
1    1002  CORREA 0412021761 KOMATSU PC 200             5   
2    1003               CORREA 10 AV 0806 C             3   
3    1004                 CORREA 10 AV 1020             9   
4    1005                 CORREA 10 AV 1165             0   

   CONSUMO_MENSUAL_PROMEDIO  TIEMPO_REPOSICION CATEGORIA  
0                         3                 30    CORREA  
1                         1                 30    CORREA  
2                         2                 30    CORREA  
3                         2                 30    CORREA  
4                         1                 30    CORREA  


In [44]:
# 2. Preprocesar los datos
# Verificar que las columnas necesarias estén presentes
columnas_necesarias = {'CODIGO', 'NOMBRE_REPUESTO', 'STOCK_ACTUAL', 'CONSUMO_MENSUAL_PROMEDIO','TIEMPO_REPOSICION', 'CATEGORIA'}
if not columnas_necesarias.issubset(df.columns):
    raise ValueError("El archivo no tiene las columnas requeridas")

In [45]:
# Asegurar que STOCK_ACTUAL sea numérico
df["STOCK_ACTUAL"] = pd.to_numeric(df["STOCK_ACTUAL"], errors='coerce').fillna(0)

In [46]:
from groq import Groq

# Inicializar el cliente de Groq con tu API key
cliente = Groq(api_key="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")

In [48]:
# 3. Definir reglas de stock mínimo (mínimo 1 unidad)
df["STOCK_MÍNIMO"] = 1

In [49]:
# 4. Construir el prompt para Groq
prompt = """
Dado el siguiente dataset de repuestos y fluidos para mantenimiento de maquinaria pesada y vehículos livianos,
analiza el stock actual y determina qué elementos requieren ajustes en el stock mínimo.

Los datos tienen la siguiente estructura:
ID | NOMBRE_REPUESTO | STOCK_ACTUAL | CATEGORÍA | STOCK_MÍNIMO

Aquí están los primeros 5 elementos:
{}

Requisitos:
1. Identificar elementos con stock inferior al mínimo recomendado.
2. Priorizar categorías críticas como FILTRO, ACEITE, COMBUSTIBLE.
3. Generar recomendaciones en formato de texto y tabla.

Responde en formato estructurado.
""".format(df.head().to_string(index=False))


In [51]:
# Crear la solicitud para obtener la respuesta de la IA usando Groq
respuesta = cliente.chat.completions.create(
    model="llama3-8b-8192",
    messages=[{"role": "system", "content": "Eres un experto en análisis de stock."},
              {"role": "user", "content": prompt}]
)

# Mostrar la respuesta del modelo
print(respuesta.choices[0].message.content)

**Análisis de Stock**

**Elementos con stock inferior al mínimo recomendado**

Después de revisar el dataset, hemos encontrado los siguientes elementos con stock inferior al mínimo recomendado:

| ID | NOMBRE_REPUESTO | STOCK_ACTUAL | STOCK_MÍNIMO |
| --- | --- | --- | --- |
| 1005 | CORREA 10 AV 1165 | 0 | 1 |

El repuesto CORREA 10 AV 1165 tiene un stock actual de 0 unidades, lo que es inferior al stock mínimo recomendado de 1 unidad.

**Categorías críticas**

Entre las categorías presentes en el dataset, se han identificado las siguientes como críticas:

* FILTRO (no se presenta en el dataset)
* ACEITE (no se presenta en el dataset)
* COMBUSTIBLE (no se presenta en el dataset)

Sin embargo, es importante considerar que la categoría de CORREAS puede ser considerada crítica, ya que se refiere a componentes esenciales para el mantenimiento de la maquinaria pesada y vehículos livianos.

**Recomendaciones**

En base a los resultados anteriores, se recomiendan las siguientes acciones:

1.

#Análisis de stock faltante de la categoría Filtro

In [71]:
# Definir el stock mínimo
STOCK_MINIMO = 1

#Identificar códigos con stock mínimo insuficiente por categoría
def identificar_faltantes_por_categoria(df, categoria):
    filtro = (df['CATEGORIA'] == categoria) & (df['STOCK_ACTUAL'] < STOCK_MINIMO)
    return df[filtro]

#Generar tabla con los faltantes críticos
def generar_tabla_faltantes(df):
    faltantes = df[df['STOCK_ACTUAL'] == 0]
    return faltantes

#Ejemplo de ejecución
categoria_elegida = "FILTRO"  # Puedes cambiarlo dinámicamente
faltantes_categoria = identificar_faltantes_por_categoria(df, categoria_elegida)
faltantes_criticos = generar_tabla_faltantes(df)

display(faltantes_criticos)


,CODIGO,NOMBRE_REPUESTO,STOCK_ACTUAL,CONSUMO_MENSUAL_PROMEDIO,TIEMPO_REPOSICION,CATEGORIA,STOCK_MÍNIMO
4,1005,CORREA 10 AV 1165,0,1,30,CORREA,1
18,1019,CORREA 19X1840 (429554) CAT 561,0,1,30,CORREA,1
39,1040,CORREA 4 VIAS 45 V X 1700,0,1,30,CORREA,1
41,1042,CORREA 5H7448,0,2,30,CORREA,1
56,1057,CORREA 8PK1830 GOOD YEAR,0,1,30,CORREA,1
...,...,...,...,...,...,...,...
1473,2474,AGUA Y SOLUCION ANTICONGELANTE (-21°C) CP THERM,0,1,30,ANTICONGELANTE,1
1481,2482,GAS OIL GRADO 1,0,3,30,GAS OIL,1
1501,2502,LUBRICANTE CARDAN 08201/4-00,0,3,30,LUBRICANTE,1
1504,2505,LUBRICANTE MOBIL POLYREX EM,0,2,30,LUBRICANTE,1


In [73]:
prompt2 = f"""
Analiza el siguiente listado de repuestos con bajo stock en la categoría {categoria_elegida}.
Genera un listado para poder realizar la compra de los faltantes.
{faltantes_categoria.to_string(index=False)}
"""

# Crear la solicitud para obtener la respuesta de la IA usando Groq
respuesta = cliente.chat.completions.create(
    model="llama3-8b-8192",
    messages=[{"role": "system", "content": "Eres un experto en análisis de stock."},
              {"role": "user", "content": prompt2}]
)
# Mostrar la respuesta del modelo
print(respuesta.choices[0].message.content)

**Listado de Repuestos con Stock Bajo para la Categoría FILTRO:**

Después de analizar el listado de repuestos, he identificado los siguientes elementos que hay que reponer debido a que el stock es bajo:

1. CODIGO 1276: FILTRO 1° DE GASIOL 5S7645 CAT D6C (Stock Actual: 0)
2. CODIGO 1310: FILTRO ACEITE 1R0739 CAT XXX XXX (Stock Actual: 0)
3. CODIGO 1320: FILTRO ACEITE 3283655 CAT (Stock Actual: 0)
...
45. CODIGO 2345: FILTRO TRANSMISION 3416643 CAT 924 H (Stock Actual: 0)

En total, hay 45 repuestos con stock bajo que necesitan ser reemplazados. Es importante revisar y cancelar cualquier orden pendiente y proceder a reponer estos elementos en la cantidad necesaria para mantener un stock minimum seguro.

Espero que esta información sea útil. Si necesitas algo más, no dudes en preguntar.


In [ ]:
#Análisis de stock faltante de la categoría Correa

In [74]:
# Definir el stock mínimo
STOCK_MINIMO = 1

#Identificar códigos con stock mínimo insuficiente por categoría
def identificar_faltantes_por_categoria(df, categoria):
    correa = (df['CATEGORIA'] == categoria) & (df['STOCK_ACTUAL'] < STOCK_MINIMO)
    return df[correa]

#Generar tabla con los faltantes críticos
def generar_tabla_faltantes(df):
    faltantes = df[df['STOCK_ACTUAL'] == 0]
    return faltantes

#Ejemplo de ejecución
categoria_elegida = "CORREA"  # Puedes cambiarlo dinámicamente
faltantes_categoria = identificar_faltantes_por_categoria(df, categoria_elegida)
faltantes_criticos = generar_tabla_faltantes(df)

display(faltantes_criticos)


,CODIGO,NOMBRE_REPUESTO,STOCK_ACTUAL,CONSUMO_MENSUAL_PROMEDIO,TIEMPO_REPOSICION,CATEGORIA,STOCK_MÍNIMO
4,1005,CORREA 10 AV 1165,0,1,30,CORREA,1
18,1019,CORREA 19X1840 (429554) CAT 561,0,1,30,CORREA,1
39,1040,CORREA 4 VIAS 45 V X 1700,0,1,30,CORREA,1
41,1042,CORREA 5H7448,0,2,30,CORREA,1
56,1057,CORREA 8PK1830 GOOD YEAR,0,1,30,CORREA,1
...,...,...,...,...,...,...,...
1473,2474,AGUA Y SOLUCION ANTICONGELANTE (-21°C) CP THERM,0,1,30,ANTICONGELANTE,1
1481,2482,GAS OIL GRADO 1,0,3,30,GAS OIL,1
1501,2502,LUBRICANTE CARDAN 08201/4-00,0,3,30,LUBRICANTE,1
1504,2505,LUBRICANTE MOBIL POLYREX EM,0,2,30,LUBRICANTE,1


In [75]:
prompt3 = f"""
Analiza el siguiente listado de repuestos con bajo stock en la categoría {categoria_elegida}.
Genera un listado para poder realizar la compra de los faltantes.
{faltantes_categoria.to_string(index=False)}
"""

# Crear la solicitud para obtener la respuesta de la IA usando Groq
respuesta = cliente.chat.completions.create(
    model="llama3-8b-8192",
    messages=[{"role": "system", "content": "Eres un experto en análisis de stock."},
              {"role": "user", "content": prompt3}]
)
# Mostrar la respuesta del modelo
print(respuesta.choices[0].message.content)

Basado en el listado de repuestos con bajo stock en la categoría CORREA, he generado un listado para realizar la compra de los faltantes:

**LISTADO DE REPUESTOS CON BAJO STOCK**

1. 1005 - CORREA 10 AV 1165 - STOCK_ACTUAL: 0 - CONSUMO_MENSUAL_PROMEDIO: 1 - TIEMPO_REPOSICION: 30
2. 1019 - CORREA 19X1840 (429554) CAT 561 - STOCK_ACTUAL: 0 - CONSUMO_MENSUAL_PROMEDIO: 1 - TIEMPO_REPOSICION: 30
3. 1040 - CORREA 4 VIAS 45 V X 1700 - STOCK_ACTUAL: 0 - CONSUMO_MENSUAL_PROMEDIO: 1 - TIEMPO_REPOSICION: 30
4. 1042 - CORREA 5H7448 - STOCK_ACTUAL: 0 - CONSUMO_MENSUAL_PROMEDIO: 2 - TIEMPO_REPOSICION: 30
5. 1057 - CORREA 8PK1830 GOOD YEAR - STOCK_ACTUAL: 0 - CONSUMO_MENSUAL_PROMEDIO: 1 - TIEMPO_REPOSICION: 30
6. 1063 - CORREA A/A 11451134 IVECO EUROCARGO - STOCK_ACTUAL: 0 - CONSUMO_MENSUAL_PROMEDIO: 2 - TIEMPO_REPOSICION: 30
7. 1066 - CORREA A34 - STOCK_ACTUAL: 0 - CONSUMO_MENSUAL_PROMEDIO: 1 - TIEMPO_REPOSICION: 30
8. 1085 - CORREA ALTERN (DAYCO 8PK-1420) FORDCARGO - STOCK_ACTUAL: 0 - CONSUMO_MENSU